In [38]:
import pandas as pd
import numpy as np
from sklearn import set_config
import sklearn.metrics as metrics
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit

In [54]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [2]:
set_config(display='diagram')

In [55]:
def canonizer(data, test_size=0.1, nb_coef=20, random_state=42, cross_validation=False, kernel='rbf'):
    
    df_results = pd.DataFrame()
    
    if cross_validation == True:
        pipe = make_pipeline(StandardScaler(), SVC(kernel=kernel, probability=True))
        cv_results = cross_validate(pipe, data.drop(['canonicity'], axis=1), data['canonicity'], cv=cv)
        return pipe, cv_results
    
    else:
        pipe = make_pipeline(StandardScaler(), SVC(kernel=kernel, probability=True))   

        X_train, X_test, y_train, y_test = train_test_split(data.drop(['canonicity'], axis=1), data['canonicity'], test_size=test_size, random_state=random_state)        

        pipe.fit(X_train, y_train)
        
        print(metrics.classification_report(y_test, pipe.predict(X_test)))
        
        canonicity_predictions = pipe.predict(X_test)

        #r2 = r2_score(y_test, canonicity_predictions)
        #mae = mean_absolute_error(y_test, canonicity_predictions)
        #mse = mean_squared_error(y_test, canonicity_predictions)
        #print("r2 : %0.1f%% " % (r2 * 100))
        #print("mean_absolute_error : ", mae)
        #print("mean_squared_error : ", mse)
        #print("root mean_squared_error : ", np.sqrt(mse))
        
        if kernel == 'linear':
            coefs = pipe.named_steps['svc'].coef_
            plot_coefficients(*coefs, data.columns, nb_coef)
            
        df_results['metadata'] = y_test
        df_results['proba canon'] = pipe.predict_proba(X_test)[:,0]
        df_results['proba non-canon'] = pipe.predict_proba(X_test)[:,1]
        df_results['prediction']= pipe.predict(X_test)
    
        df_results['accord'] = [True if row['metadata'] == row['prediction'] else False for index, row in df_results.iterrows()]
    
    
    return pipe, df_results

# BoW features 

In [3]:
df_main = pd.read_csv(r'../data/01_chunks_BoW_features.csv')
df_main.set_index("index", inplace = True)
df_main = df_main.replace(np.nan, 0)

In [23]:
def retrieve_canonicity(index_name):
    list_canonicity = []
    for titre in index_name:
        temp_tag = titre.split('_')[3]
        list_canonicity.append(temp_tag)
    return list_canonicity

In [24]:
list_canonicity = retrieve_canonicity(df_main.index)

In [26]:
df_main['canonicity'] = list_canonicity

In [34]:
pipe_main, df_results_main = canonizer(df_main)

              precision    recall  f1-score   support

       canon       0.90      0.85      0.88      1385
    noncanon       0.89      0.93      0.91      1791

    accuracy                           0.90      3176
   macro avg       0.90      0.89      0.89      3176
weighted avg       0.90      0.90      0.90      3176



In [35]:
pipe_main

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('normalizer', Normalizer()), ('svc', SVC(probability=True))])

In [52]:
pipe_test, df_results_test = canonizer(df_main)

              precision    recall  f1-score   support

       canon       0.90      0.87      0.88       689
    noncanon       0.90      0.92      0.91       899

    accuracy                           0.90      1588
   macro avg       0.90      0.89      0.90      1588
weighted avg       0.90      0.90      0.90      1588



In [53]:
pipe_test

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(probability=True))])

# LDA features 

# DBoW neural embeddings features 